# Visualisation de population ( Bordeaux Métropole )

## 0. Introduction
Ce projet vise à estimer la population d'une zone géographique spécifique à partir de diverses sources de données et méthodes. L'objectif principal est de fournir une estimation fiable en combinant des informations spatiales (carreaux IRIS, contours IRIS, carreaux Filosofi) et des données spécifiques aux logements (adresses résidentielles, diagnostics de performance énergétique - DPE).

### Sources de données utilisées
1. **Carreaux IRIS :**  
   - Petites unités géographiques définies par l'INSEE contenant des données de population.

2. **Contours IRIS :**  
   - Polygones définissant les limites des zones IRIS.
   - Utilisés pour estimer la population et les logements en fonction des ratios d'intersection.

3. **Carreaux Filosofi (200m) :**  
   - Données à l'échelle fine (200m) contenant des informations sur les ménages et les individus.
   - Fournissent une granularité supplémentaire pour l'estimation.

4. **Adresses résidentielles :**  
   - Récupérées via l'API Overpass et enrichies avec l'API Nominatim.
   - Utilisées comme une source complémentaire pour identifier les logements dans la zone.

5. **Données DPE (Diagnostics de Performance Énergétique) :**  
   - Fournissent des informations sur la surface habitable et permettent de distinguer les maisons des appartements.
   - Utilisées pour affiner l'estimation du type de logements et leur population.

## 1. Carte interactive

### Fonctionnalités principales

1. **Ajout de marqueurs :**
   - Cliquez sur la carte pour ajouter jusqu'à deux marqueurs.
   - Une fois les deux marqueurs placés, un rectangle est automatiquement tracé entre eux pour délimiter la zone sélectionnée.
   - Les coordonnées des marqueurs sont enregistrées comme points de départ (`start`) et d'arrivée (`end`).

2. **Visualisation des carreaux :**
   - Les carreaux géographiques filtrés (ex. : Filosofi ou IRIS) sont affichés sur la carte :
     - **Rouge** pour les carreaux Filosofi.
     - **Vert** pour les contours IRIS.
   - Chaque carreau est dessiné comme un polygone avec une opacité pour faciliter la lecture.

3. **Affichage des points d'adresse :**
   - Des points circulaires sont affichés pour représenter les adresses.
   - La couleur du cercle indique :
     - **Rouge** : Une ou plusieurs adresses sont associées à des DPE (Diagnostics de Performance Énergétique).
     - **Bleu** : Aucun DPE n'est trouvé pour l'adresse.
   - Un nombre peut être associé à chaque cercle et affiché dans une `Popup` visible au clic.

4. **Réinitialisation :**
   - Un bouton "Reset" est disponible pour réinitialiser la carte.
   - Ce bouton supprime tous les marqueurs, rectangles, polygones et points affichés sur la carte.
   - Les coordonnées de départ et d'arrivée (`start` et `end`) sont également réinitialisées.


### Étapes d'utilisation

1. **Sélection de la zone :**
   - Cliquez sur deux points sur la carte pour définir une zone rectangulaire.
   - Vérifiez les coordonnées des points définis (`start` et `end`).

2. **Affichage des données :**
   - Chargez les carreaux ou contours filtrés pour qu'ils soient affichés dans leur couleur respective.

3. **Ajout de points d'adresse :**
   - Ajoutez des points représentant les adresses sur la carte.
   - Les points affichent des informations contextuelles sur les DPE ou les données associées.

4. **Réinitialisation :**
   - Cliquez sur le bouton "Reset" pour vider la carte et redémarrer.

In [ ]:
from ipyleaflet import Map, Rectangle, Marker, Polygon, CircleMarker, Popup
from ipywidgets import widgets, HTML
from pyproj import Transformer
import asyncio

class Coord:
    def __init__(self, lat=0, lon=0, name=None):
        self.lat = lat
        self.lon = lon
        self.name = name
    
    def set(self, lat, lon):
        self.lat = lat
        self.lon = lon
        print(f"{self.name} Coordinates set to {self.lat}, {self.lon}")
    
    def __str__(self):
        return f"{self.name}: {self.lat}, {self.lon}" if self.name else f"{self.lat}, {self.lon}"
    
class MapView:
    def __init__(self):
        self.map = Map(center=[44.8378, -0.5792], zoom=13)
        self.rectangles = []
        self.markers = []
        self.polygons = []
        self.circle_markers = []
        self.map.on_interaction(self.handle_click_map)
        self.map.layout.height = '700px'
        display(self.map)
    
    def add_rectangle(self):
            bounds = (self.markers[0].location, self.markers[1].location)
            rectangle = Rectangle(bounds=bounds, color="blue", fill_opacity=0.3)
            self.rectangles.append(rectangle)
            self.map.add_layer(rectangle)

    def set_coords(self):
        global start, end
        start.set(self.markers[0].location[0], self.markers[0].location[1])
        end.set(self.markers[1].location[0], self.markers[1].location[1])

    def add_marker(self, lat, lon):
        if len(self.markers) == 2:
            return
        self.markers.append(Marker(location=(lat, lon)))
        self.map.add_layer(self.markers[-1])
        if len(self.markers) == 2:
            self.add_rectangle()
            self.set_coords()

    
    def clear(self):
        for rect in self.rectangles:
            self.map.remove_layer(rect)
        self.rectangles = []
        for marker in self.markers:
            self.map.remove_layer(marker)
        self.markers = []
        for polygon in self.polygons:
            self.map.remove_layer(polygon)
        self.polygons = []
        for circle_marker in self.circle_markers:
            self.map.remove_layer(circle_marker)
        self.circle_markers = []

    
    def handle_click_map(self, **kwargs):
        if kwargs.get('type') == 'click':
            lat, lon = kwargs.get('coordinates')
            self.add_marker(lat, lon)
            

    def add_carreaux(self, filtered_carreaux, color="red"):
        # Reprojeter les géométries en EPSG:4326
        filtered_carreaux = filtered_carreaux.to_crs("EPSG:4326")
    
        # Parcourir les géométries des carreaux
        for _, row in filtered_carreaux.iterrows():
            # Vérifier si la géométrie est un polygone
            if row["geometry"].geom_type == "Polygon":
                coords = [(y, x) for x, y in row["geometry"].exterior.coords]
                # Créer un polygone avec la couleur spécifiée et une faible opacité
                polygon = Polygon(
                    locations=coords,
                    color=color,
                    fill_opacity=0.3,
                )
                # Ajouter le polygone à la carte
                self.polygons.append(polygon)
                self.map.add_layer(polygon)
    
    def add_iris(self, filtered_iris_contours):
        # Reprojeter les géométries en EPSG:4326 pour la carte
        filtered_iris_contours = filtered_iris_contours.to_crs("EPSG:4326")
        
        # Parcourir les géométries des IRIS
        for _, row in filtered_iris_contours.iterrows():
            if row["geometry"].geom_type == "MultiPolygon":
                # Gérer un MultiPolygon
                for polygon_geom in row["geometry"].geoms:
                    coords = [(y, x) for x, y in polygon_geom.exterior.coords]
                    polygon = Polygon(
                        locations=coords,
                        color="green",
                        fill_color="green",
                        fill_opacity=0.3,
                    )
                    self.polygons.append(polygon)
                    self.map.add_layer(polygon)
                    
    def add_point(self, lat, lon, number=0, color="black"):
        # Créer un marqueur circulaire avec la couleur spécifiée
        circle_marker = CircleMarker(
            location=(lat, lon),
            radius=5,  # Taille du point
            color=color,  # Couleur de la bordure
            fill_color=color,  # Couleur de remplissage
            fill_opacity=0.8,  # Opacité du remplissage
        )
        self.circle_markers.append(circle_marker)
        self.map.add_layer(circle_marker)
        
        text_html = HTML(
            value=f'<div style="font-size: 12px; color: black; text-align: center; font-weight: bold;">{number}</div>'
        )
        text_popup = Popup(location=(lat, lon), child=text_html, close_button=False, auto_close=False)
        circle_marker.popup = text_popup
    
class ResetButton:
    def __init__(self):
        self.button = widgets.Button(description="Reset")
        self.button.on_click(self.handle_click_button)
        display(self.button)
    
    def handle_click_button(self, b):
        global map_view
        map_view.clear()
        print("Map cleared.")

        global start, end
        start = Coord(name="Start")
        end = Coord(name="End")
        print("Start and End coordinates reset.")

start = Coord(name="Start")
end = Coord(name="End")

map_view = MapView()
reset_button = ResetButton()



## 2. Traitement et définition des zones géographiques

### Définition des zones

1. **Zone EPSG:2154 :**
   - Utilisée pour les analyses IRIS et Filosofi, adaptée à la France.
   - Créée avec `to_epsg_2154()` :
     ```python
     zone_bbox = geo.to_epsg_2154()
     ```

2. **Bounding Box EPSG:4326 :**
   - Utilisée pour les adresses (Overpass, Nominatim).
   - Créée avec `to_epsg_4326_bbox()` :
     ```python
     address_bbox = geo.to_epsg_4326_bbox()
     ```

### Filtrage et intersections

La fonction `filter_and_calculate_intersections` :
- Filtre les entités intersectant la zone.
- Calcule les intersections, les surfaces et le ratio.


In [2]:
import geopandas as gpd
from shapely.geometry import box

# Classe pour la gestion des zones géographiques
class GeographicBBox:
    def __init__(self, start, end):
        self.start = start
        self.end = end
    def to_epsg_2154(self):
        transformer = Transformer.from_crs("EPSG:4326", "EPSG:2154", always_xy=True)
        new_start = transformer.transform(self.start.lon, self.start.lat)
        new_end = transformer.transform(self.end.lon, self.end.lat)
        bounding_box = box(new_start[0], new_start[1], new_end[0], new_end[1])
        return gpd.GeoDataFrame({"geometry": [bounding_box]}, crs="EPSG:2154")
    def to_epsg_4326_bbox(self):
        south = min(self.start.lat, self.end.lat)
        west = min(self.start.lon, self.end.lon)
        north = max(self.start.lat, self.end.lat)
        east = max(self.start.lon, self.end.lon)
        return f"{south},{west},{north},{east}"

# Fonction pour filtrer et calculer les intersections de la zone avec les data geopandas en entrée
def filter_and_calculate_intersections(data, zone_bbox):
    data_filtered = data[data.intersects(zone_bbox.union_all())].copy()
    data_filtered["intersection"] = data_filtered["geometry"].intersection(zone_bbox.union_all())
    data_filtered["intersection_area"] = data_filtered["intersection"].area
    data_filtered["carreau_area"] = data_filtered["geometry"].area
    data_filtered["intersection_ratio"] = data_filtered["intersection_area"] / data_filtered["carreau_area"]
    return data_filtered

geo = GeographicBBox(start, end)

zone_bbox = geo.to_epsg_2154()

address_bbox = geo.to_epsg_4326_bbox()


## 3. Extraction et traitement des données IRIS

### 1. Carreaux IRIS
- **Données sélectionnées :**
  - `id_carreau_1km` : Identifiant unique de chaque carreau de 1 km².
  - `pop` : Population associée au carreau.
  - `intersection_ratio` : Ratio entre l'aire d'intersection du carreau avec la zone et l'aire totale du carreau.
- Ces données permettent de quantifier la population au sein de la zone géographique étudiée.
- **Visualisation :** Les carreaux IRIS sont affichés en **jaune** sur la carte.
- **Note :** Si le DataFrame est vide, cela signifie qu'aucun carreau ne chevauche la zone définie.

### 2. Contours IRIS
- **Données sélectionnées :**
  - `code_iris` : Code identifiant utilisé pour la jointure avec les données démographique IRIS.
  - `intersection_ratio` : Ratio entre l'aire d'intersection du contour IRIS avec la zone et l'aire totale de l'IRIS.
- Ces données identifient les IRIS partiellement ou entièrement incluses dans la zone étudiée.
- **Visualisation :** Les contours IRIS sont affichés en **vert** sur la carte.
- **Note :** Si le DataFrame est vide, cela signifie qu'aucun contour IRIS ne chevauche la zone.

### 3. Données démographiques IRIS
- **Données de population (`iris_pop`) :**
  - `IRIS` : Code identifiant de l'IRIS.
  - `P21_POP` : Population totale en 2021.
  - `P21_PMEN` : Population des ménages.
  - `P21_PHORMEN` : Population hors ménage.
  - Ces données sont enrichies avec `intersection_ratio` pour refléter la part de population concernée par la zone.
  
- **Données de logement (`iris_log`) :**
  - `IRIS` : Code identifiant de l'IRIS.
  - `P21_LOG` : Nombre total de logements.
  - `P21_MAISON` : Nombre de maisons.
  - `P21_APPART` : Nombre d'appartements.
  - Ces données sont également enrichies avec `intersection_ratio` pour estimer la part des logements concernés.


In [ ]:
import pandas as pd

# Charger et traiter les carreaux IRIS
carreaux_iris = gpd.read_file("./data/carreaux_1km_met.gpkg")
carreaux_iris_filtered = filter_and_calculate_intersections(carreaux_iris, zone_bbox)
map_view.add_carreaux(carreaux_iris_filtered, color="yellow")
data_carreaux_iris = carreaux_iris_filtered[[ "id_carreau_1km", "pop", "intersection_ratio"]]


# Charger et traiter les contours IRIS
iris_contours = gpd.read_file(
    "data/CONTOURS-IRIS_3-0__GPKG_LAMB93_FXX_2024-01-01/CONTOURS-IRIS/1_DONNEES_LIVRAISON_2024-12-00163/CONTOURS-IRIS_3-0_GPKG_LAMB93_FXX-ED2024-01-01/contours-iris.gpkg"
)
filtered_iris_contours = filter_and_calculate_intersections(iris_contours, zone_bbox)
map_view.add_iris(filtered_iris_contours)
# Charger les données population et logement pour les IRIS
iris_pop = pd.read_csv(
    "data/base-ic-evol-struct-pop-2021_csv/base-ic-evol-struct-pop-2021.CSV",
    sep=";", encoding="utf-8", usecols=["IRIS", "P21_POP", "P21_PMEN", "P21_PHORMEN"], low_memory=False
)
iris_log = pd.read_csv(
    "data/base-ic-logement-2021_csv/base-ic-logement-2021.CSV",
    sep=";", encoding="utf-8", usecols=["IRIS", "P21_LOG", "P21_MAISON", "P21_APPART"], low_memory=False
)
# Filtrer les données de population et logement
data_iris_pop = iris_pop[iris_pop["IRIS"].isin(filtered_iris_contours["code_iris"])]
data_iris_log = iris_log[iris_log["IRIS"].isin(filtered_iris_contours["code_iris"])]
# Ajouter le champ intersection_ratio aux données démographiques
data_iris_pop = data_iris_pop.merge(
    filtered_iris_contours[["code_iris", "intersection_ratio"]],
    left_on="IRIS",
    right_on="code_iris",
    how="left"
)
data_iris_log = data_iris_log.merge(
    filtered_iris_contours[["code_iris", "intersection_ratio"]],
    left_on="IRIS",
    right_on="code_iris",
    how="left"
)

# Affichage des résultats
print(data_carreaux_iris)
print(data_iris_pop)
print(data_iris_log)

## 4. Extraction et traitement des données Filosofi

### 1. Carreaux Filosofi (200m)
- **Données sélectionnées :**
  - `idcar_200m` : Identifiant unique de chaque carreau de 200m².
  - `ind` : Indice de revenu médian par unité géographique.
  - `men` : Nombre de ménages associés au carreau.
  - `intersection_ratio` : Ratio entre l'aire d'intersection du carreau avec la zone et l'aire totale du carreau.
- Ces données permettent une analyse fine des revenus et ménages à petite échelle (200m²).
- **Visualisation :** Les carreaux Filosofi sont affichés en **rouge** sur la carte.

In [ ]:
carreaux_200m_filosofi = gpd.read_file("./data/Filosofi2019_carreaux_200m_gpkg/carreaux_200m_met.gpkg")
carreaux_200m_filosofi_filtered = filter_and_calculate_intersections(carreaux_200m_filosofi, zone_bbox)
map_view.add_carreaux(carreaux_200m_filosofi_filtered, color="red")
data_carreaux_200m_filosofi = carreaux_200m_filosofi_filtered[["idcar_200m", "ind", "men", "intersection_ratio"]]
print(data_carreaux_200m_filosofi)

## 5. Extraction et traitement des adresses et DPE (diagnostique de performance énergétique)

### Étapes principales

1. **Récupération des adresses avec Overpass :**
   - L'API Overpass est utilisée pour récupérer toutes les entités contenant des informations sur les bâtiments résidentiels et les adresses (`addr:housenumber`) dans la zone géographique définie.
   - Les coordonnées géographiques (`lat`, `lon`) des entités sont extraites.

2. **Filtrage des adresses résidentielles :**
   - Les résultats d'Overpass sont filtrés pour ne conserver que les bâtiments résidentiels, tels que :
     - Les bâtiments marqués comme `residential`, `house`, ou `apartments`.
     - Les adresses avec un numéro de maison (`addr:housenumber`), tout en excluant les entités commerciales (`shop`, `office`, etc.).

3. **Enrichissement avec Nominatim :**
   - L'API Nominatim est utilisée pour enrichir chaque adresse avec des informations détaillées :
     - `housenumber` : Numéro de la maison.
     - `street` : Rue.
     - `city` : Ville.
   - Ces informations permettent d'obtenir une adresse complète pour chaque point (`lat`, `lon`).

4. **Normalisation des adresses :**
   - Chaque composant de l'adresse est normalisé pour uniformiser le format :
     - Conversion en majuscules.
     - Suppression des caractères spéciaux (accents, ponctuation).
     - Élimination des espaces superflus.


### Association avec la base de données DPE

1. **Chargement des données DPE :**
   - Les données des diagnostics de performance énergétique (DPE) sont chargées depuis un fichier CSV.
   - Une colonne `formatted_address` est ajoutée, qui combine les informations de la rue, du type de voie, et de la commune pour former une adresse unique.

2. **Filtrage des DPE par adresses enrichies :**
   - Chaque adresse normalisée issue de Nominatim est utilisée pour filtrer les données DPE correspondantes :
     - Recherche basée sur le numéro de maison (`housenumber`), la rue (`street`), et la ville (`city`).
   - Les DPE correspondants sont ajoutés à une liste.

3. **Visualisation des adresses sur la carte :**
   - Chaque adresse enrichie est affichée sur une carte avec un marqueur :
     - **Rouge** si des DPE sont trouvés pour cette adresse.
     - **Bleu** si aucun DPE n'est associé.

4. **Fusion des résultats DPE :**
   - Les DPE filtrés sont combinés en un tableau final, où les collones sont enrichies pour inclure des informations détaillées :
     - `adresse_complete` : Adresse complète avec bâtiment, étage, et porte.
     - `surface_habitable` : Surface habitable du logement.
     - `consommation_energie` : Consommation énergétique du logement.



In [ ]:
import requests
import sys
from concurrent.futures import ThreadPoolExecutor

def get_addresses_with_overpass(bbox):
    """
    Utilise l'API Overpass pour récupérer les adresses avec "addr:housenumber" dans une bbox.
    
    :param bbox: String, bbox au format "south,west,north,east"
    :return: Liste des adresses trouvées
    """
    overpass_url = "http://overpass-api.de/api/interpreter"
    
    query = f"""
    [out:json];
    (
      way["building"="residential"]({bbox});
      way["building"="house"]({bbox});
      node["addr:housenumber"]({bbox});
    );
    out body;
    """

    try:
        response = requests.get(overpass_url, params={"data": query}, timeout=30)
        response.raise_for_status()
        data = response.json()
        return data.get("elements", [])

    except requests.exceptions.RequestException as e:
        print(f"Erreur API Overpass : {e}")
        return []
    
def filter_residential_addresses(elements):
    residential_addresses = []
    for element in elements:
        tags = element.get("tags", {})
        
        is_residential = (
            tags.get("building") in ["residential", "house", "apartments"] or
            tags.get("addr:housenumber") and not any(key in tags for key in ["shop", "amenity", "office", "cuisine"])
        )
        if not is_residential:
            continue  
        
        address = {
            "lat": element.get("lat"),
            "lon": element.get("lon")
        }
        residential_addresses.append(address)
    
    return residential_addresses

def fetch_address_details(address):
    """
    Fonction pour enrichir une adresse via l'API Nominatim.
    :param address: Dictionnaire contenant "lat" et "lon".
    :return: Dictionnaire enrichi ou None en cas d'erreur.
    """
    nominatim_url = "https://nominatim.openstreetmap.org/reverse"
    lat = address.get("lat")
    lon = address.get("lon")
    
    if lat is None or lon is None:
        return None

    params = {
        "lat": lat,
        "lon": lon,
        "format": "json",
        "addressdetails": 1
    }
    headers = {
        "User-Agent": "MyPythonApp/1.0 (contact@example.com)"
    }
    
    try:
        response = requests.get(nominatim_url, params=params, headers=headers, timeout=10)
        response.raise_for_status()
        data = response.json()
        address_details = data.get("address", {})
        enriched_address = {
            "housenumber": address_details.get("house_number", ""),
            "street": address_details.get("road", ""),
            "city": address_details.get("city", address_details.get("town", "")),
            "lat": lat,
            "lon": lon
        }
        return enriched_address
    except requests.exceptions.RequestException as e:
        print(f"Erreur API Nominatim pour lat={lat}, lon={lon} : {e}")
        return None

def enrich_addresses_with_nominatim(addresses, max_workers=10):
    """
    Enrichit une liste d'adresses en parallèle avec l'API Nominatim.
    
    :param addresses: Liste de dictionnaires contenant "lat" et "lon".
    :param max_workers: Nombre maximal de threads pour le traitement parallèle.
    :return: Liste de dictionnaires enrichis.
    """
    enriched_addresses = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(fetch_address_details, address): address for address in addresses}
        for i, future in enumerate(futures):
            sys.stdout.write(f"\rEnrichissement de l'adresse {i + 1}/{len(addresses)}")
            sys.stdout.flush()
            result = future.result()
            if result:
                enriched_addresses.append(result)
    print("\n")
    return enriched_addresses

def normalize_address(address):
    """Normalise une adresse : majuscules, suppression des espaces superflus, et remplacement des caractères spéciaux."""
    address = address.upper().strip()  # Tout en majuscules et suppression des espaces
    address = address.replace("É", "E").replace("È", "E").replace("À", "A")  # Retirer les accents
    address = address.replace(",", "").replace(".", "")  # Supprimer les caractères inutiles
    return address

# Extraire les addresses résidentielles avec Overpass et filtrer les résultats pour les adresses résidentielles
residential_addresses = filter_residential_addresses(get_addresses_with_overpass(geo.to_epsg_4326_bbox()))
print("Adresses résidentielles trouvées :", len(residential_addresses))

# Enrichir les adresses avec les détails complets
residential_addresses = enrich_addresses_with_nominatim(residential_addresses)

# Normaliser les adresses pour la comparaison
for address in residential_addresses:
    address['housenumber'] = normalize_address(address['housenumber'])
    address['street'] = normalize_address(address['street'])
    address['city'] = normalize_address(address['city'])
    
import pandas as pd

def format_address(row):
    numero = str(row['numero_rue']).strip() if not pd.isna(row['numero_rue']) else ''
    voie = str(row['type_voie']).strip() if not pd.isna(row['type_voie']) else ''
    rue = str(row['nom_rue']).strip() if not pd.isna(row['nom_rue']) else ''
    commune = str(row['commune']).strip() if not pd.isna(row['commune']) else ''
    # Combiner les éléments pour former l'adresse finale
    adresse = ' '.join(filter(None, [numero, voie, rue, commune]))
    return normalize_address(adresse)

# Charger les données DPE
dpe_data = pd.read_csv("data/dpe-33.csv", sep=",", encoding="utf-8", low_memory=False)

# Ajouter la colonne des adresses formatées
dpe_data['formatted_address'] = dpe_data.apply(format_address, axis=1)

filtered_dpe_data_list = []
# Filtrer pour chaque adresse normalisée
for address in residential_addresses:
    filtered_data = dpe_data[
        dpe_data['formatted_address'].str.contains(address["housenumber"], na=False) &
        dpe_data['formatted_address'].str.contains(address["street"], na=False) &
        dpe_data['formatted_address'].str.contains(address["city"], na=False)
    ]
    nb_dpe = len(filtered_data)
    map_view.add_point(address["lat"], address["lon"], nb_dpe, color="red" if nb_dpe > 0 else "blue")
    filtered_dpe_data_list.append(filtered_data)

data_dpe = pd.DataFrame()
# Combiner tous les résultats filtrés
if len(filtered_dpe_data_list) > 0:
    filtered_dpe_data = pd.concat(filtered_dpe_data_list, ignore_index=True)
    # Fusionner les colonnes avec un séparateur, par exemple ", "
    filtered_dpe_data['adresse_complete'] = (
        filtered_dpe_data['formatted_address'].fillna('') +
        filtered_dpe_data['batiment'].apply(lambda x: f", Bâtiment: {x}" if pd.notna(x) else '') +
        filtered_dpe_data['etage'].apply(lambda x: f", Étage: {x}" if pd.notna(x) else '') +
        filtered_dpe_data['porte'].apply(lambda x: f", Porte: {x}" if pd.notna(x) else '')
    )
    data_dpe = filtered_dpe_data[['adresse_complete', 'surface_habitable', 'consommation_energie']]

# Afficher les résultats
nb_residential_addresses = len(residential_addresses)
print("Nombre d'adresses résidentielles enrichies :", nb_residential_addresses)
print("Nombre de dpe trouvées :", len(data_dpe))
print("Données DPE filtrées :")
display(data_dpe)


## 6. Estimation des logements et du taux de maisons

Cette partie du code a pour objectif d'estimer le nombre total de logements et le taux de maisons dans une zone donnée à partir de plusieurs sources de données.

### Étapes de l'estimation :

1. **Contours IRIS :**
   - À partir des contours IRIS, on estime :
     - Le nombre total de logements (**`log_iris`**) en multipliant la variable `P21_LOG` (nombre de logements IRIS) par le ratio d'intersection avec la zone étudiée.
     - Le nombre de maisons (**`maison_iris`**) en multipliant la variable `P21_MAISON` (nombre de maisons IRIS) par le ratio d'intersection.

2. **Carreaux Filosofi (200m) :**
   - À partir des carreaux Filosofi, on estime le nombre de logements (**`log_carreaux_200m_filosofi`**) en multipliant la variable `men` (nombre de ménages) par le ratio d'intersection.

3. **DPE (Diagnostics de Performance Énergétique) :**
   - Les DPE sont utilisés pour classifier les logements :
     - Une maison est identifiée si sa surface dépasse un seuil donné (**`maison_threshold`**, par défaut 100 m²).
     - On calcule le nombre total de maisons estimées à partir des DPE.

4. **Combinaison des données :**
   - Le nombre total de logements est calculé en combinant :
     - 40% des logements estimés via les contours IRIS.
     - 40% des logements estimés via les carreaux Filosofi.
     - 20% des adresses résidentielles enrichies.
   - Le taux de maisons est calculé en combinant :
     - Le taux de maisons dérivé des contours IRIS (pondération basée sur leur fiabilité relative).
     - Le taux de maisons dérivé des DPE, pondéré dynamiquement selon la couverture des DPE dans la zone étudiée.

### Résultats estimés :
- **Nombre total de logements estimés :**
  Combinaison des sources IRIS, Filosofi et adresses enrichies.
- **Taux de maisons estimé :**
  Ratio des maisons par rapport aux logements totaux, en tenant compte des données des DPE et des IRIS.

In [ ]:
# Estimation logements par les contours IRIS
def estimate_logements_iris(data_iris_log):
    """
    Estime les logements et les maisons à partir des contours IRIS.

    :param data_iris_log: DataFrame contenant les données des contours IRIS.
    :return: Tuple (log_iris, maison_iris).
    """
    data_iris_log = data_iris_log.copy()
    data_iris_log["estimated_log"] = data_iris_log["P21_LOG"] * data_iris_log["intersection_ratio"]
    data_iris_log["estimated_maison"] = data_iris_log["P21_MAISON"] * data_iris_log["intersection_ratio"]
    log_iris = data_iris_log["estimated_log"].sum()
    maison_iris = data_iris_log["estimated_maison"].sum()
    return log_iris, maison_iris

# Estimation logements par les carreaux Filosofi 200m
def estimate_logements_filosofi(data_carreaux_200m_filosofi):
    """
    Estime les logements à partir des carreaux Filosofi 200m.

    :param data_carreaux_200m_filosofi: DataFrame contenant les données des carreaux Filosofi 200m.
    :return: Nombre total de logements estimés.
    """
    data_carreaux_200m_filosofi = data_carreaux_200m_filosofi.copy()
    data_carreaux_200m_filosofi["estimated_log"] = (
        data_carreaux_200m_filosofi["men"] * data_carreaux_200m_filosofi["intersection_ratio"]
    )
    return data_carreaux_200m_filosofi["estimated_log"].sum()

# Classification des DPE
def classify_housing_by_surface(data_dpe, maison_threshold=100):
    """
    Compte le nombre de maisons à partir des surfaces des DPE, sans modifier le DataFrame.

    :param data_dpe: DataFrame contenant les DPE.
    :param maison_threshold: Seuil de surface pour classifier une maison.
    :return: Nombre total de maisons estimées.
    """
    if len(data_dpe) == 0:
        return 0
    nb_maisons = (data_dpe["surface_habitable"] > maison_threshold).sum()
    return nb_maisons

# Estimation du nombre total de logements et du taux de maisons
def estimate_housing_and_maison_ratio(
    log_iris, maison_iris, log_carreaux_200m_filosofi, nb_residential_addresses, nb_dpe, nb_maisons_dpe
):
    """
    Estime le nombre total de logements et le taux de maisons en combinant plusieurs sources.

    :param log_iris: Nombre de logements estimés par les contours IRIS.
    :param maison_iris: Nombre de maisons estimées par les contours IRIS.
    :param log_carreaux_200m_filosofi: Nombre de logements estimés par les carreaux Filosofi 200m.
    :param nb_residential_addresses: Nombre total d'adresses résidentielles enrichies.
    :param nb_dpe: Nombre total de DPE disponibles.
    :param nb_maisons_dpe: Nombre de maisons estimées à partir des DPE.
    :return: Dictionnaire contenant le nombre total de logements et le taux de maisons.
    """
    total_logements = (
        0.4 * log_iris +
        0.4 * log_carreaux_200m_filosofi +
        0.2 * nb_residential_addresses
    )
    taux_dpe = nb_dpe / nb_residential_addresses if nb_residential_addresses > 0 else 0
    if taux_dpe < 0:
        taux_dpe = 0
    if taux_dpe <= 1:
        poids_dpe = taux_dpe * 0.5
    else:
        poids_dpe = min(0.5 + (taux_dpe - 1) * 0.25, 0.8)

    poids_iris = 1.0 - poids_dpe
    print("Taux DPE :", taux_dpe)
    print("Poids DPE :", poids_dpe)
    taux_maison_combined = (
        poids_iris * (maison_iris / log_iris if log_iris > 0 else 0) +
        poids_dpe * (nb_maisons_dpe / nb_dpe if nb_dpe > 0 else 0)
    )
    
    print ("Taux maison combined :", taux_maison_combined)
    print ("Maison iris :", maison_iris)
    print ("Log iris :", log_iris)
    
    return {
        "total_logements_estimes": round(total_logements),
        "taux_maison_estime": taux_maison_combined,
    }

# Données d'entrée
log_iris, maison_iris = estimate_logements_iris(data_iris_log)
log_carreaux_200m_filosofi = estimate_logements_filosofi(data_carreaux_200m_filosofi)
nb_maisons_dpe = classify_housing_by_surface(data_dpe, maison_threshold=100)

# Calcul final
log_estimation = estimate_housing_and_maison_ratio(
    log_iris=log_iris,
    maison_iris=maison_iris,
    log_carreaux_200m_filosofi=log_carreaux_200m_filosofi,
    nb_residential_addresses=nb_residential_addresses,
    nb_dpe=len(data_dpe),
    nb_maisons_dpe=nb_maisons_dpe
)

# Affichage des résultats
print("Estimation iris logements :", log_iris)
print("Estimation carreaux 200m Filosofi logements :", log_carreaux_200m_filosofi)
print("---------------------------------")
print("Estimation du nombre total de logements :", log_estimation["total_logements_estimes"])
print("")
print("Estimation iris maisons :", maison_iris)
print("---------------------------------")
print("Estimation du taux de maisons :", log_estimation["taux_maison_estime"])

## 7. Estimation de la population

Cette partie du code a pour but d'estimer la population totale d'une zone en combinant plusieurs sources de données et méthodes.

### Étapes de l'estimation :

1. **Population estimée par les carreaux IRIS :**
   - Utilisation de la population des carreaux IRIS (**`pop`**) pondérée par le ratio d'intersection avec la zone étudiée (**`intersection_ratio`**).
   - Résultat : Population estimée à partir des carreaux IRIS.

2. **Population estimée par les contours IRIS :**
   - Utilisation de la population IRIS (**`P21_POP`**) pondérée par le ratio d'intersection avec la zone.
   - Résultat : Population estimée à partir des contours IRIS.

3. **Population estimée par les carreaux Filosofi (200m) :**
   - Utilisation des individus des carreaux Filosofi (**`ind`**) pondérée par le ratio d'intersection.
   - Résultat : Population estimée à partir des carreaux Filosofi.

4. **Combinaison des données directes et des logements estimés :**
   - **Population directe :** Moyenne pondérée des populations estimées par :
     - 30% des carreaux IRIS.
     - 30% des contours IRIS.
     - 40% des carreaux Filosofi.
   - **Population via les logements :**
     - Utilisation du nombre de logements estimés.
     - Calcul des habitants :
       - **Maisons :** Nombre moyen d'habitants par maison (par défaut 2.5).
       - **Appartements :** Nombre moyen d'habitants par appartement (par défaut 1.8).

5. **Estimation finale :**
   - Moyenne pondérée entre la population directe (50%) et celle estimée via les logements (50%).

### Résultats estimés :
- **Population directe estimée :** Résultant des données IRIS et Filosofi.
- **Population via les logements estimée :** Calculée à partir des logements et des taux de maisons.
- **Population totale estimée :** Moyenne pondérée des deux précédentes.


In [ ]:
def estimate_population_carreaux_iris(data_carreaux_iris):
    """
    Estime la population à partir des carreaux IRIS en utilisant l'intersection ratio.

    :param data_carreaux_iris: DataFrame contenant les données des carreaux IRIS.
    :return: Population estimée.
    """
    data_carreaux_iris = data_carreaux_iris.copy()
    data_carreaux_iris["estimated_pop"] = data_carreaux_iris["pop"] * data_carreaux_iris["intersection_ratio"]
    return data_carreaux_iris["estimated_pop"].sum()

def estimate_population_iris(data_iris_pop):
    """
    Estime la population à partir des contours IRIS en utilisant l'intersection ratio.

    :param data_iris_pop: DataFrame contenant les données des contours IRIS.
    :return: Population estimée.
    """
    data_iris_pop = data_iris_pop.copy()
    data_iris_pop["estimated_pop"] = data_iris_pop["P21_POP"] * data_iris_pop["intersection_ratio"]
    return data_iris_pop["estimated_pop"].sum()

def estimate_population_carreaux_filosofi(data_carreaux_200m_filosofi):
    """
    Estime la population à partir des carreaux Filosofi 200m en utilisant l'intersection ratio.

    :param data_carreaux_200m_filosofi: DataFrame contenant les données des carreaux Filosofi 200m.
    :return: Population estimée.
    """
    data_carreaux_200m_filosofi = data_carreaux_200m_filosofi.copy()
    data_carreaux_200m_filosofi["estimated_pop"] = (
        data_carreaux_200m_filosofi["ind"] * data_carreaux_200m_filosofi["intersection_ratio"]
    )
    return data_carreaux_200m_filosofi["estimated_pop"].sum()

def estimate_population_from_data(data, avg_per_maison=2.5, avg_per_appart=1.8):
    """
    Estime la population totale en combinant les données directes et les logements.

    :param data: Dictionnaire contenant les données nécessaires pour l'estimation.
    :param avg_per_maison: Nombre moyen d'habitants par maison.
    :param avg_per_appart: Nombre moyen d'habitants par appartement.
    :return: Dictionnaire contenant les estimations de population.
    """
    # Population directe
    pop_direct = (
        0.3 * data["pop_carreaux_iris"] +
        0.3 * data["pop_iris"] +
        0.4 * data["pop_carreaux_200m_filosofi"]
    )

    # Population estimée à partir des logements
    nb_maisons = data["total_logements_estimes"] * data["taux_maison_estime"]
    nb_appartements = data["total_logements_estimes"] * (1 - data["taux_maison_estime"])
    pop_from_logements = (nb_maisons * avg_per_maison) + (nb_appartements * avg_per_appart)

    # Estimation combinée
    total_population = (
        0.5 * pop_direct +  # Pondération pour la population directe
        0.5 * pop_from_logements  # Pondération pour la population via les logements
    )

    return {
        "pop_direct": pop_direct,
        "pop_from_logements": pop_from_logements,
        "population_totale": total_population,
    }

# Calcul des populations
pop_carreaux_iris = estimate_population_carreaux_iris(data_carreaux_iris)
pop_iris = estimate_population_iris(data_iris_pop)
pop_carreaux_200m_filosofi = estimate_population_carreaux_filosofi(data_carreaux_200m_filosofi)

# Données pour l'estimation finale
data = {
    "pop_carreaux_iris": pop_carreaux_iris,
    "pop_iris": pop_iris,
    "pop_carreaux_200m_filosofi": pop_carreaux_200m_filosofi,
    "total_logements_estimes": log_estimation["total_logements_estimes"],
    "taux_maison_estime": log_estimation["taux_maison_estime"],
}

# Estimation finale
population_estimation = estimate_population_from_data(data)

# Affichage des résultats
print("Estimation de la population :")
print(f"Population carreaux IRIS : {pop_carreaux_iris:.0f} habitants")
print(f"Population IRIS : {pop_iris:.0f} habitants")
print(f"Population carreaux Filosofi 200m : {pop_carreaux_200m_filosofi:.0f} habitants")
print("---------------------------------")
print(f"Population directe : {population_estimation['pop_direct']:.0f} habitants")
print(f"Population estimée par les logements : {population_estimation['pop_from_logements']:.0f} habitants")
print(f"Population totale estimée : {population_estimation['population_totale']:.0f} habitants")


## 8. Validation du resultat

Pour valider notre méthode, nous utilisons un carreau Filosofi 200 m connu, sélectionné parmi les carreaux qui intersectent la zone d'étude. Ce carreau est traité comme s'il s'agissait d'une zone inconnue. Nous appliquons l'intégralité de notre démarche d'estimation à ce carreau, puis comparons les résultats obtenus avec ses valeurs réelles. Cela permet d'évaluer la précision du modèle et d'ajuster ses paramètres si nécessaire.

In [ ]:
# Récupérer un carreau Filosofi 200m pour la validation
validation_data = carreaux_200m_filosofi[carreaux_200m_filosofi.intersects(zone_bbox.union_all())]

validation_start = Coord("validation_start")
validation_end = Coord("validation_start")

# Définir un transformer pour convertir les coordonnées
transformer_to_wgs84 = Transformer.from_crs("EPSG:2154", "EPSG:4326", always_xy=True)

# Extraire le premier carreau Filosofi de validation
premier_carreau = validation_data.iloc[0]  # Premier carreau dans le GeoDataFrame

# Obtenir les coordonnées géométriques (bounding box du carreau)
bbox = premier_carreau.geometry.bounds
min_x, min_y, max_x, max_y = bbox

# Convertir les coordonnées Lambert 93 en WGS 84
min_lon, min_lat = transformer_to_wgs84.transform(min_x, min_y)
max_lon, max_lat = transformer_to_wgs84.transform(max_x, max_y)

# Définir les coordonnées de validation
validation_start.set(min_lat, min_lon)
validation_end.set(max_lat, max_lon)

validation_geo = GeographicBBox(validation_start, validation_end)

# Traiter les carreaux IRIS
validation_carreaux_iris_filtered = filter_and_calculate_intersections(carreaux_iris, validation_geo.to_epsg_2154())
validation_data_carreaux_iris = validation_carreaux_iris_filtered[[ "id_carreau_1km", "pop", "intersection_ratio"]]

# Traiter les contours IRIS
validation_filtered_iris_contours = filter_and_calculate_intersections(iris_contours, validation_geo.to_epsg_2154())

# Filtrer les données de population et logement
validation_data_iris_pop = iris_pop[iris_pop["IRIS"].isin(validation_filtered_iris_contours["code_iris"])]
validation_data_iris_log = iris_log[iris_log["IRIS"].isin(validation_filtered_iris_contours["code_iris"])]
# Ajouter le champ intersection_ratio aux données démographiques
validation_data_iris_pop = validation_data_iris_pop.merge(
    validation_filtered_iris_contours[["code_iris", "intersection_ratio"]],
    left_on="IRIS",
    right_on="code_iris",
    how="left"
)
validation_data_iris_log = validation_data_iris_log.merge(
    validation_filtered_iris_contours[["code_iris", "intersection_ratio"]],
    left_on="IRIS",
    right_on="code_iris",
    how="left"
)

# Traiter le carreaux Filosofi 200m
validation_carreaux_200m_filosofi_filtered = filter_and_calculate_intersections(carreaux_200m_filosofi, validation_geo.to_epsg_2154())
validation_data_carreaux_200m_filosofi = validation_carreaux_200m_filosofi_filtered[["idcar_200m", "ind", "men", "intersection_ratio"]]

# Extraire les addresses résidentielles avec Overpass et filtrer les résultats pour les adresses résidentielles
validation_residential_addresses = filter_residential_addresses(get_addresses_with_overpass(validation_geo.to_epsg_4326_bbox()))
print("Adresses résidentielles trouvées :", len(validation_residential_addresses))

# Enrichir les adresses avec les détails complets
validation_residential_addresses = enrich_addresses_with_nominatim(validation_residential_addresses)

# Normaliser les adresses pour la comparaison
for address in validation_residential_addresses:
    address['housenumber'] = normalize_address(address['housenumber'])
    address['street'] = normalize_address(address['street'])
    address['city'] = normalize_address(address['city'])

# Traitement des DPE
validation_filtered_dpe_data_list = []
# Filtrer pour chaque adresse normalisée
for address in validation_residential_addresses:
    filtered_data = dpe_data[
        dpe_data['formatted_address'].str.contains(address["housenumber"], na=False) &
        dpe_data['formatted_address'].str.contains(address["street"], na=False) &
        dpe_data['formatted_address'].str.contains(address["city"], na=False)
    ]
    nb_dpe = len(filtered_data)
    validation_filtered_dpe_data_list.append(filtered_data)

validation_data_dpe = pd.DataFrame()
# Combiner tous les résultats filtrés
if len(validation_filtered_dpe_data_list) > 0:
    validation_filtered_dpe_data = pd.concat(validation_filtered_dpe_data_list, ignore_index=True)
    # Fusionner les colonnes avec un séparateur, par exemple ", "
    validation_filtered_dpe_data['adresse_complete'] = (
        validation_filtered_dpe_data['formatted_address'].fillna('') +
        validation_filtered_dpe_data['batiment'].apply(lambda x: f", Bâtiment: {x}" if pd.notna(x) else '') +
        validation_filtered_dpe_data['etage'].apply(lambda x: f", Étage: {x}" if pd.notna(x) else '') +
        validation_filtered_dpe_data['porte'].apply(lambda x: f", Porte: {x}" if pd.notna(x) else '')
    )
    validation_data_dpe = validation_filtered_dpe_data[['adresse_complete', 'surface_habitable', 'consommation_energie']]

# Afficher les résultats
validation_nb_residential_addresses = len(validation_residential_addresses)


### Estimation

# Données d'entrée
validation_log_iris, validation_maison_iris = estimate_logements_iris(validation_data_iris_log)
validation_log_carreaux_200m_filosofi = estimate_logements_filosofi(validation_data_carreaux_200m_filosofi)
validation_nb_maisons_dpe = classify_housing_by_surface(validation_data_dpe, maison_threshold=100)

# Calcul final
validation_log_estimation = estimate_housing_and_maison_ratio(
    log_iris=validation_log_iris,
    maison_iris=validation_maison_iris,
    log_carreaux_200m_filosofi=validation_log_carreaux_200m_filosofi,
    nb_residential_addresses=validation_nb_residential_addresses,
    nb_dpe=len(validation_data_dpe),
    nb_maisons_dpe=validation_nb_maisons_dpe
)

# Affichage des résultats
print("Estimation iris logements :", validation_log_iris)
print("Estimation carreaux 200m Filosofi logements :", validation_log_carreaux_200m_filosofi)
print("---------------------------------")
print("Estimation du nombre total de logements :", validation_log_estimation["total_logements_estimes"])
print("")
print("Estimation iris maisons :", validation_maison_iris)
print("---------------------------------")
print("Estimation du taux de maisons :", validation_log_estimation["taux_maison_estime"])
print("---------------------------------")
print("---------------------------------")


# Calcul des populations
validation_pop_carreaux_iris = estimate_population_carreaux_iris(validation_data_carreaux_iris)
validation_pop_iris = estimate_population_iris(validation_data_iris_pop)
validation_pop_carreaux_200m_filosofi = estimate_population_carreaux_filosofi(validation_data_carreaux_200m_filosofi)

# Données pour l'estimation finale
validation_data = {
    "pop_carreaux_iris": validation_pop_carreaux_iris,
    "pop_iris": validation_pop_iris,
    "pop_carreaux_200m_filosofi": validation_pop_carreaux_200m_filosofi,
    "total_logements_estimes": validation_log_estimation["total_logements_estimes"],
    "taux_maison_estime": validation_log_estimation["taux_maison_estime"],
}

# Estimation finale
validation_population_estimation = estimate_population_from_data(validation_data)

# Affichage des résultats
print(f"Population carreaux IRIS : {validation_pop_carreaux_iris:.0f} habitants")
print(f"Population IRIS : {validation_pop_iris:.0f} habitants")
print(f"Population carreaux Filosofi 200m : {validation_pop_carreaux_200m_filosofi:.0f} habitants")
print("---------------------------------")
print(f"Population directe : {validation_population_estimation['pop_direct']:.0f} habitants")
print(f"Population estimée par les logements : {validation_population_estimation['pop_from_logements']:.0f} habitants")
print(f"Population totale estimée : {validation_population_estimation['population_totale']:.0f} habitants")
print(f"--------------------------------")

erreur_pourcentage = ((validation_population_estimation['population_totale'] - validation_pop_carreaux_200m_filosofi) 
                          / validation_pop_carreaux_200m_filosofi if validation_pop_carreaux_200m_filosofi > 0 else 0) * 100

print(f"Errer avec filosofi 200m : {erreur_pourcentage:.2f}%")

